# Chat with prompty

## 0. Install dependent packages

In [1]:
%%capture --no-stderr
%pip install promptflow-devkit

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 44.0.2 which is incompatible.



     ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.0 MB 6.3 MB/s eta 0:00:02
     --- ------------------------------------ 0.7/7.0 MB 8.7 MB/s eta 0:00:01
     ---- ----------------------------------- 0.8/7.0 MB 8.8 MB/s eta 0:00:01
     -------- ------------------------------- 1.5/7.0 MB 8.4 MB/s eta 0:00:01
     ---------- ----------------------------- 1.9/7.0 MB 8.0 MB/s eta 0:00:01
     ----------- ---------------------------- 2.1/7.0 MB 7.3 MB/s eta 0:00:01
     --------------- ------------------------ 2.7/7.0 MB 8.5 MB/s eta 0:00:01
     ------------------ --------------------- 3.2/7.0 MB 8.8 MB/s eta 0:00:01
     -------------------- ------------------- 3.6/7.0 MB 8.9 MB/s eta 0:00:01
     ---------------------- ----------------- 4.0/7.0 MB 8.7 MB/s eta 0:00:01
     ------------------------ --------------- 4.3/7.0 MB 8.8 MB/s eta 0:00:01
     -------------------------- ------------- 4.7/7.0 MB 8.8 MB/s eta 

In [2]:
%pip install promptflow promptflow-tools

     ---------------------------------------- 0.0/53.6 kB ? eta -:--:--
     ---------------------------------------- 53.6/53.6 kB 1.4 MB/s eta 0:00:00
  Using cached google_search_results-2.4.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


## 1. Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [1]:
with open("prompty/chat.prompty") as fin:
    print(fin.read())

---
name: Chat Prompt
description: A basic prompt that uses the chat API to answer questions with chat_history
model:
    api: chat
    configuration:
        type: azure_openai
        connection: my_azure_open_ai_connection
        azure_deployment: gpt-4-0125-Preview
    parameters:
        max_tokens: 256
        temperature: 0.2

inputs:
    question:
        type: string
    chat_history:
        type: list
        default: []
sample:
    question: What is the meaning of life?
    chat_history: []

---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly, 
and in a personable manner using markdown and even add some personal flair with appropriate emojis.

{% for item in chat_history %}
{{item.role}}:
{{item.content}}
{% endfor %}

user:
{{question}}


### Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

We need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

In [ ]:
#%pip install keyrings.alt

In [2]:
from promptflow.client import PFClient
from promptflow.connections import AzureOpenAIConnection, OpenAIConnection

from promptflow.entities import AzureOpenAIConnection
client = PFClient()
# Initialize an AzureOpenAIConnection object
connection = AzureOpenAIConnection(
    name="my_azure_open_ai_connection",
    api_key="XXX",
    api_base="XXX",
)
# Create the connection, note that api_key will be scrubbed in the returned result
result = client.connections.create_or_update(connection)
print(result)

print(connection)

auth_mode: key
name: my_azure_open_ai_connection
module: promptflow.connections
created_date: '2024-07-04T01:25:55.056628'
last_modified_date: '2024-08-21T07:59:58.201822'
type: azure_open_ai
api_key: '******'
api_base: https://azuremlopenai.openai.azure.com/
api_type: azure
api_version: '2024-02-01'

auth_mode: key
name: my_azure_open_ai_connection
module: promptflow.connections
type: azure_open_ai
api_key: '******'
api_base: https://azuremlopenai.openai.azure.com/
api_type: azure
api_version: '2024-02-01'



In [4]:
conn_name = "my_azure_open_ai_connection"
conn = client.connections.get(name=conn_name)
print("using this connection :",conn_name)

using this connection : my_azure_open_ai_connection


### Execute prompty as function

In [5]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load("prompty/chat.prompty")
# execute the flow as function
question = "What is the capital of France?"
result = f(question=question)
result

"The capital of France is Paris! 🇫🇷✨ It's not just the political capital but also a global center for art, fashion, gastronomy, and culture. A truly iconic city!"

You can override connection with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [7]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration


# override configuration with created connection in AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    connection="my_azure_open_ai_connection", azure_deployment="gpt-4o"
)

# override openai connection with OpenAIModelConfiguration
# configuration = OpenAIModelConfiguration(
#     connection=connection,
#     model="gpt-3.5-turbo"
# )

override_model = {
    "configuration": configuration,
}

# load prompty as a flow
f = Prompty.load("prompty/chat.prompty", model=override_model)
# execute the flow as function
question = "What is the capital of France?"
result = f(question=question)
result

'The capital of France is Paris! 🇫🇷✨'

### Visualize trace by using start_trace

In [20]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Prompt flow service has started...


Re-run below cell will collect a trace in trace UI.

In [21]:
# rerun the function, which will be recorded in the trace
result = f(question=question)
result

'The capital of France is Paris! 🇫🇷✨'

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=AzureOpenAI_Advanced&uiTraceId=0x73a5f4ff53faf77ee686f6a84d7556b4


### Eval the result 

In this example, we will use a prompt that determines whether a chat conversation contains an apology from the assistant.

In [9]:
eval_prompty = "prompty/apology.prompty"

with open(eval_prompty) as fin:
    print(fin.read())

---
name: Apology Prompt
description: A prompt that determines whether a chat conversation contains an apology from the assistant
model:
  api: chat
  configuration:
    type: azure_openai
    connection: my_azure_open_ai_connection
    azure_deployment: gpt-4o
  parameters:
    temperature: 0.2
    response_format: { "type": "json_object" }
inputs: 
  question:
    type: string
  answer:
    type: string
  messages:
    type: list
outputs:
  apology:
    type: string
sample: ${file:sample.json}
---

system:
You are an AI tool that determines if, in a chat conversation, the assistant apologized, like say sorry.
Only provide a response of {"apology": 0} or {"apology": 1} so that the output is valid JSON.
Give a apology of 1 if apologized in the chat conversation.

Here are some examples of chat conversations and the correct response:

**Example 1**
user: Where can I get my car fixed?
assistant: I'm sorry, I don't know that. Would you like me to look it up for you?
result:
{"apology": 1}

Note: the eval flow returns a `json_object`.

In [11]:
# load prompty as a flow
eval_flow = Prompty.load(eval_prompty)
# execute the flow as function
result = eval_flow(question=question, answer=result, messages=[])
result

{'apology': 0}

In [9]:
!pf service stop

Prompt flow service stop on 127.0.0.1:23333.


# Batch run with multi-line data


In [1]:
from promptflow.client import PFClient

flow = "./prompty/chat.prompty"  # path to the prompty file
data = "./prompty/data.jsonl"  # path to the data file

# create run with the flow and data
pf = PFClient()
base_run = pf.run(
    flow=flow,
    data=data,
    column_mapping={
        "question": "${data.question}",
        "chat_history": "${data.chat_history}",
    },
    stream=True,
)

[2024-08-21 08:22:47 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run prompty_20240821_082247_217704, log path: /home/azureuser/.promptflow/.runs/prompty_20240821_082247_217704/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=prompty_20240821_082247_217704
2024-08-21 08:22:55 +0000 3217081 execution.bulk     INFO     Process 3217104 terminated.
2024-08-21 08:22:55 +0000 3217081 execution.bulk     WARNING  Process 3217120 had been terminated.
2024-08-21 08:22:55 +0000 3217081 execution.bulk     WARNING  Process 3217109 had been terminated.
2024-08-21 08:22:47 +0000 3217002 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-08-21 08:22:47 +0000 3217002 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3}.
2024-08-21 08:22:49 +0000 3217002 execution.bulk     INFO     Process name(ForkProcess-2:3)-Process id(3217120)-Line number(0) start execution.
2024-08-21 08:22:49 +0000 3217002 execution.bulk     INFO     Process name(ForkProcess-2:1)-Pro

In [2]:
details = pf.get_details(base_run)
details

,inputs.question,inputs.chat_history,inputs.line_number,outputs.output
0,What's chat-GPT?,[],0,Chat-GPT is like a digital buddy that's super ...
1,How many questions did I ask?,[],1,You've asked one question so far! 🎉 Got any mo...
2,Summarize our conversation?,"[{'role': 'user', 'content': 'where is the nea...",2,Of course! 😊 You asked about the nearest coffe...


In [3]:
details.head(3)['outputs.output'].values

array(["Chat-GPT is like a digital buddy that's super good at chatting! 🤖✨ Developed by OpenAI, it's a type of AI (Artificial Intelligence) that's designed to have conversations with humans. Whether you need help with homework, want to know the weather, or just feel like chatting about your favorite TV show, Chat-GPT is there to keep the conversation going. It's based on a model called GPT (Generative Pre-trained Transformer), which is just a fancy way of saying it's really good at understanding and generating human-like text. So, it's like having a chat with a friend who knows a ton of stuff! 📚💬",
       "You've asked one question so far! 🎉 Got any more for me? 😊",
       "Of course! 😊 You asked about the nearest coffee shop, but since I couldn't access your location, I couldn't provide a specific answer. Then, you inquired about Azure ML, and I briefly described it as a cloud-based platform for machine learning provided by Microsoft Azure. It's designed to help users build, train, an